In [1]:
import math as m
import numpy as np
import random as r
import matplotlib.pyplot as plt
import sys

In [2]:
import torch
from torch import nn
from torch import optim
from torch.optim.lr_scheduler import MultiStepLR

In [3]:
from nflows.flows.base import Flow
from nflows.distributions.uniform import BoxUniform
from nflows.transforms.base import CompositeTransform
from nflows.transforms.autoregressive import MaskedPiecewiseRationalQuadraticAutoregressiveTransform
from nflows.transforms.autoregressive import MaskedPiecewiseQuadraticAutoregressiveTransform
from nflows.transforms.permutations import ReversePermutation
from nflows.transforms.permutations import RandomPermutation
from nflows.transforms.splines.rational_quadratic import rational_quadratic_spline
from torch.utils.tensorboard import SummaryWriter

In [4]:
import subprocess
import time
import os
from copy import deepcopy
import math as m
import gc

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [6]:
device = torch.device("cuda:0")
#device = torch.device("cpu")

In [7]:
flow = torch.load("archive/final_flow_model.pt", map_location=device)
flow.eval()

Flow(
  (_transform): CompositeTransform(
    (_transforms): ModuleList(
      (0): RandomPermutation()
      (1): MaskedPiecewiseRationalQuadraticAutoregressiveTransform(
        (autoregressive_net): MADE(
          (initial_layer): MaskedLinear(in_features=20, out_features=500, bias=True)
          (blocks): ModuleList(
            (0): MaskedFeedforwardBlock(
              (linear): MaskedLinear(in_features=500, out_features=500, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (1): MaskedFeedforwardBlock(
              (linear): MaskedLinear(in_features=500, out_features=500, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (final_layer): MaskedLinear(in_features=500, out_features=580, bias=True)
        )
      )
      (2): RandomPermutation()
      (3): MaskedPiecewiseRationalQuadraticAutoregressiveTransform(
        (autoregressive_net): MADE(
          (initial_layer): MaskedLinear(

In [8]:
n_samples = int(1e6)
n_sample_size = int(1e4)

steps = int(n_samples/n_sample_size)

generated = None
with torch.no_grad():
    i = 0
    while generated is None or generated.shape[0] < n_samples: 
        try:
            new_data = flow.sample(n_sample_size).cpu().numpy()
        except AssertionError:
            continue
        i += 1
        print("Sampled {} datapoints ({} / {})...".format(n_sample_size, i, steps))
        if generated is None:
            generated = new_data
        else:
            generated = np.vstack((generated, new_data))

Sampled 10000 datapoints (1 / 100)...
Sampled 10000 datapoints (2 / 100)...
Sampled 10000 datapoints (3 / 100)...
Sampled 10000 datapoints (4 / 100)...
Sampled 10000 datapoints (5 / 100)...
Sampled 10000 datapoints (6 / 100)...
Sampled 10000 datapoints (7 / 100)...
Sampled 10000 datapoints (8 / 100)...
Sampled 10000 datapoints (9 / 100)...
Sampled 10000 datapoints (10 / 100)...
Sampled 10000 datapoints (11 / 100)...
Sampled 10000 datapoints (12 / 100)...
Sampled 10000 datapoints (13 / 100)...
Sampled 10000 datapoints (14 / 100)...
Sampled 10000 datapoints (15 / 100)...
Sampled 10000 datapoints (16 / 100)...
Sampled 10000 datapoints (17 / 100)...
Sampled 10000 datapoints (18 / 100)...
Sampled 10000 datapoints (19 / 100)...
Sampled 10000 datapoints (20 / 100)...
Sampled 10000 datapoints (21 / 100)...
Sampled 10000 datapoints (22 / 100)...
Sampled 10000 datapoints (23 / 100)...
Sampled 10000 datapoints (24 / 100)...
tensor([], device='cuda:0')
tensor([nan, nan, nan, nan, nan, nan, nan, na

In [9]:
np.savetxt('data/generated_samples.csv', generated, delimiter=',')

In [10]:
generated

array([[0.33343816, 0.4798584 , 0.04498715, ..., 0.14674628, 0.70641494,
        0.6842612 ],
       [0.46345973, 0.5871932 , 0.8373328 , ..., 0.97240883, 0.18121597,
        0.89961946],
       [0.5081225 , 0.6781636 , 0.8968967 , ..., 0.9638471 , 0.8135269 ,
        0.3345296 ],
       ...,
       [0.4912488 , 0.57304806, 0.60279834, ..., 0.7877582 , 0.91348153,
        0.60721934],
       [0.50237477, 0.6297763 , 0.94187945, ..., 0.42585826, 0.462759  ,
        0.33246768],
       [0.5037309 , 0.5469548 , 0.7929161 , ..., 0.48206145, 0.40181604,
        0.71073836]], dtype=float32)